In [1]:
import copy

# 外皮情報LV2からLV3へのコンバート

## I. 関数

### 1. 主たる居室・その他の居室・非居室に接する外皮の部位の面積・長さ

#### 1) 面積

$$ A_{evlp,mr,i} = \frac{ A_{f,mr} }{ A_{f,total} } A_{evlp,i} $$

$$ A_{evlp,or,i} = \frac{ A_{f,or} }{ A_{f,total} } A_{evlp,i} $$

$$A_{evlp,nr,i} = \frac{ A_{f,total} - A_{f,mr} - A_{f,or} }{ A_{f,total}}   A_{evlp,i} $$

#### 2) 長さ

$$ l_{evlp,mr,i} = \frac{ A_{f,mr} }{ A_{f,total} } l_{evlp,i} $$

$$ l_{evlp,or,i} = \frac{ A_{f,or} }{ A_{f,total} } l_{evlp,i} $$

$$ l_{evlp,nr,i} = \frac{ A_{f,total} - A_{f,mr} - A_{f,or} }{ A_{f,total} } l_{evlp,i} $$

$A_{evlp,mr,i}$：主たる居室に接する外皮の部位（一般部位・開口部）$i$の面積, m<sup>2</sup>  
$A_{evlp,or,i}$：その他の居室に接する外皮の部位（一般部位・開口部）$i$の面積, m<sup>2</sup>  
$A_{evlp,nr,i}$：非居室に接する外皮の部位（一般部位・開口部）$i$の面積, m<sup>2</sup>  
$l_{evlp,mr,i}$：主たる居室に接する外皮の部位（熱橋）$i$の長さ, m  
$l_{evlp,or,i}$：その他の居室に接する外皮の部位（熱橋）$i$の長さ, m  
$l_{evlp,nr,i}$：非居室に接する外皮の部位（熱橋）$i$の長さ, m  
$A_{evlp,i}$ : 外皮の部位（一般部位・開口部）$i$の面積, m<sup>2</sup>  
$l_{evlp,i}$ : 外皮の部位（熱橋）$i$の面積, m  
$A_{f,total}$ : 床面積の合計, m<sup>2</sup>  
$A_{f,mr}$ : 主たる居室の床面積, m<sup>2</sup>  
$A_{f,or}$ : その他の居室の床面積, m<sup>2</sup>  

In [2]:
def get_separated_areas(a_evlp_i, a_f_mr, a_f_or, a_f_total):

    a_evlp_mr_i = a_f_mr / a_f_total * a_evlp_i
    a_evlp_or_i = a_f_or / a_f_total * a_evlp_i
    a_evlp_nr_i = (a_f_total - a_f_mr - a_f_or) / a_f_total * a_evlp_i
    
    return a_evlp_mr_i, a_evlp_or_i, a_evlp_nr_i

In [3]:
def get_separated_lengths(l_evlp_i, a_f_mr, a_f_or, a_f_total):

    l_evlp_mr_i = a_f_mr / a_f_total * l_evlp_i
    l_evlp_or_i = a_f_or / a_f_total * l_evlp_i
    l_evlp_nr_i = (a_f_total - a_f_mr - a_f_or) / a_f_total * l_evlp_i
    
    return l_evlp_mr_i, l_evlp_or_i, l_evlp_nr_i

$j=1$の時は主たる居室に接し、$j=2$の時はその他の居室に接し、$j=3$の時は非居室に接する

In [4]:
def get_space_types():
    
    return 'main_occupant_room', 'other_occupant_room', 'non_occupant_room'

### 2. 外皮の部位の分類

#### 1) 一般部

In [5]:
def get_general_parts(general_parts, common):
    
    separated_general_parts = []
    
    space_type_mr, space_type_or, space_type_nr = get_space_types()
    
    for general_part in general_parts:
        
        a_evlp_mr, a_evlp_or, a_evlp_nr = get_separated_areas(
            a_evlp_i=general_part['area'], 
            a_f_mr=common['main_occupant_room_floor_area'],
            a_f_or=common['other_occupant_room_floor_area'], 
            a_f_total=common['total_floor_area']
        )
        
        for space_type, a_evlp in zip((space_type_mr, space_type_or, space_type_nr), (a_evlp_mr, a_evlp_or, a_evlp_nr)):
            
            separated_general_parts.append(
                {
                    'name'              : general_part['name'] + '_' + space_type,
                    'general_part_type' : general_part['general_part_type'],
                    'next_space'        : general_part['next_space'],
                    'direction'         : general_part['direction'],
                    'area'              : a_evlp,
                    'space_type'        : space_type,
                    'spec'              : copy.deepcopy(general_part['spec'])
                }
            )
            
    return separated_general_parts

#### Example

In [6]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name'              : 'ceiling',
                'general_part_type' : 'ceiling',
                'next_space'        : 'outdoor', 
                'direction'         : 'top', 
                'area'              : 67.8,
                'spec'              : 'something',
            },
            {
                'name'              : 'floor',
                'general_part_type' : 'floor',
                'next_space'        : 'outdoor', 
                'direction'         : 'bottom', 
                'area'              : 67.8,
                'spec'              : 'something',
            },
            {
                'name'              : 'wall',
                'general_part_type' : 'wall',
                'next_space'        : 'outdoor', 
                'direction'         : 'n', 
                'area'              : 67.8,
                'spec'              : 'something',
            },
        ],
    }
} 

get_general_parts(d['envelope']['general_parts'], d['common'])

[{'name': 'ceiling_main_occupant_room',
  'general_part_type': 'ceiling',
  'next_space': 'outdoor',
  'direction': 'top',
  'area': 16.95,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'ceiling_other_occupant_room',
  'general_part_type': 'ceiling',
  'next_space': 'outdoor',
  'direction': 'top',
  'area': 33.9,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'ceiling_non_occupant_room',
  'general_part_type': 'ceiling',
  'next_space': 'outdoor',
  'direction': 'top',
  'area': 16.95,
  'space_type': 'non_occupant_room',
  'spec': 'something'},
 {'name': 'floor_main_occupant_room',
  'general_part_type': 'floor',
  'next_space': 'outdoor',
  'direction': 'bottom',
  'area': 16.95,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'floor_other_occupant_room',
  'general_part_type': 'floor',
  'next_space': 'outdoor',
  'direction': 'bottom',
  'area': 33.9,
  'space_type': 'other_occupant_room',
  'spec': 'somet

#### 2) 大部分がガラスで構成される窓等の開口部

In [7]:
def get_windows(windows, common):
    
    separated_windows = []
    
    space_type1, space_type2, space_type3 = get_space_types()
    
    for window in windows:
        
        area1, area2, area3 = get_separated_areas(
            a_evlp_i=window['area'], 
            a_f_mr=common['main_occupant_room_floor_area'], 
            a_f_or=common['other_occupant_room_floor_area'], 
            a_f_total=common['total_floor_area']
        )
        
        for space_type, area in zip((space_type1, space_type2, space_type3), (area1, area2, area3)):
            separated_windows.append(
                {
                    'name'       : window['name'] + '_' + space_type,
                    'next_space' : window['next_space'],
                    'direction'  : window['direction'],
                    'area'       : area,
                    'space_type' : space_type,
                    'spec'       : copy.deepcopy(window['spec'])
                }
            )
        
    return separated_windows

example

In [8]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'windows': [
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                'spec'       : 'something',
            },
            {
                'name'       : 'window_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 3.17,
                'spec'       : 'something',
            },
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                'spec'       : 'something',
            },
            {
                'name'       : 'window_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 3.17,
                'spec'       : 'something',
            }
        ]
    }
}    

get_windows(d['envelope']['windows'], d['common'])

[{'name': 'window_sw_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'sw',
  'area': 7.5625,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'window_sw_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'sw',
  'area': 15.125,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'window_sw_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'sw',
  'area': 7.5625,
  'space_type': 'non_occupant_room',
  'spec': 'something'},
 {'name': 'window_nw_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.7925,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'window_nw_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 1.585,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'window_nw_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.7925,
  'space_type': 'non_occupant_room',
  'sp

#### 3) 大部分がガラスで構成されないドア等の開口部

In [9]:
def get_doors(doors, common):
    
    separated_doors = []
    
    space_type1, space_type2, space_type3 = get_space_types()
    
    for door in doors:
        
        area1, area2, area3 = get_separated_areas(
            a_evlp_i=door['area'], 
            a_f_mr=common['main_occupant_room_floor_area'], 
            a_f_or=common['other_occupant_room_floor_area'], 
            a_f_total=common['total_floor_area']
        )
        
        for space_type, area in zip((space_type1, space_type2, space_type3), (area1, area2, area3)):

            separated_doors.append(
                {
                    'name'       : door['name'] + '_' + space_type,
                    'next_space' : door['next_space'],
                    'direction'  : door['direction'],
                    'area'       : area,
                    'space_type' : space_type,
                    'spec'       : copy.deepcopy(door['spec'])
                }
            )
        
    return separated_doors

example

In [10]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'doors': [
            {
                'name'       : 'door_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 2.52,
                'spec'       : 'something'
            },
            {
                'name'       : 'doorNE',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'area'       : 2.16,
                'spec'       : 'something'
            }
        ]
    }
}    

get_doors(d['envelope']['doors'], d['common'])

[{'name': 'door_nw_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.63,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'door_nw_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 1.26,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'door_nw_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.63,
  'space_type': 'non_occupant_room',
  'spec': 'something'},
 {'name': 'doorNE_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'ne',
  'area': 0.54,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'doorNE_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'ne',
  'area': 1.08,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'doorNE_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'ne',
  'area': 0.54,
  'space_type': 'non_occupant_room',
  'spec': 'something'}]

#### 4) 熱橋

In [11]:
def get_heatbridges(heatbridges, common):

    separated_heatbridges =[]
    
    space_type1, space_type2, space_type3 = get_space_types()
    
    for heatbridge in heatbridges:
        
        length1, length2, length3 = get_separated_lengths(
            l_evlp_i=heatbridge['length'], 
            a_f_mr=common['main_occupant_room_floor_area'], 
            a_f_or=common['other_occupant_room_floor_area'], 
            a_f_total=common['total_floor_area']
        )
        
        for space_type, length in zip((space_type1, space_type2, space_type3), (length1, length2, length3)):
            
            separated_heatbridges.append(
                {
                    'name'       : heatbridge['name'] + '_' + space_type,
                    'next_space' : heatbridge['next_space'],
                    'direction'  : heatbridge['direction'],
                    'space_type' : space_type,
                    'length'     : length,
                    'spec'       : copy.deepcopy(heatbridge['spec']),
                }
            )

    return separated_heatbridges

example

In [12]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'heatbridges': [
            {
                'name'       : 'heatbridge_ne',
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'e'],
                'length'     : 1.0,
                'spec'       : 'something',
            },
            {
                'name'       : 'heatbridge_nw',
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'w'],
                'length'     : 2.0,
                'spec'       : 'something',
            }
        ]
    }
}    

get_heatbridges(d['envelope']['heatbridges'], d['common'])

[{'name': 'heatbridge_ne_main_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'e'],
  'space_type': 'main_occupant_room',
  'length': 0.25,
  'spec': 'something'},
 {'name': 'heatbridge_ne_other_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'e'],
  'space_type': 'other_occupant_room',
  'length': 0.5,
  'spec': 'something'},
 {'name': 'heatbridge_ne_non_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'e'],
  'space_type': 'non_occupant_room',
  'length': 0.25,
  'spec': 'something'},
 {'name': 'heatbridge_nw_main_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'w'],
  'space_type': 'main_occupant_room',
  'length': 0.5,
  'spec': 'something'},
 {'name': 'heatbridge_nw_other_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'w'],
  'space_type': 'other_occupant_room',
  'length': 1.0,
  'spec': 'something'},
 {'name': 'heatbridge_nw_non_occupan

#### 5) 土間床等の外周部

In [13]:
def get_earthfloor_perimeters(earthfloor_perimeters):
    
    return [
        {
            'name'       : earthfloor_perimeter['name'],
            'next_space' : earthfloor_perimeter['next_space'],
            'direction'  : earthfloor_perimeter['direction'],
            'length'     : earthfloor_perimeter['length'],
            'space_type' : 'underfloor',
            'spec'       : copy.deepcopy(earthfloor_perimeter['spec']),
        }
        for earthfloor_perimeter in earthfloor_perimeters ]

example

In [14]:
d = {
    'earthfloor_perimeters': [
        {
            'name'       : 'earthfloor_perimeter_nw',
            'next_space' : 'outdoor',
            'direction'  : 'nw',
            'length'     : 2.43,
            'spec': 'something'
        },
        {
            'name'       : 'earthfloor_perimeter_ne',
            'next_space' : 'outdoor',
            'direction'  : 'ne',
            'length'     : 2.43,
            'spec': 'something'
        }
    ]
}    

get_earthfloor_perimeters(d['earthfloor_perimeters'])

[{'name': 'earthfloor_perimeter_nw',
  'next_space': 'outdoor',
  'direction': 'nw',
  'length': 2.43,
  'space_type': 'underfloor',
  'spec': 'something'},
 {'name': 'earthfloor_perimeter_ne',
  'next_space': 'outdoor',
  'direction': 'ne',
  'length': 2.43,
  'space_type': 'underfloor',
  'spec': 'something'}]

#### 6) 土間床等の中心部

In [15]:
def get_earthfloor_centers(earthfloor_centers):
    
    return [
        {
            'name'       : earghfloor_center['name'],
            'area'       : earghfloor_center['area'],
            'space_type' : 'underfloor'
        }
        for earghfloor_center in earthfloor_centers ]

example

In [16]:
d = {
    'earthfloor_centers': [
        { 'area': 5.0, 'name': 'other' },
        { 'area': 5.0, 'name': 'entrance' }
    ]
}    

get_earthfloor_centers(d['earthfloor_centers'])

[{'name': 'other', 'area': 5.0, 'space_type': 'underfloor'},
 {'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor'}]

## II. 統合

In [17]:
def convert(d):
    
    common_lv2 = d['common']
    envelope_lv2 = d['envelope']
    
    common_lv3 = copy.deepcopy(common_lv2)
    envelope_lv3 = {}
    
    if 'general_parts' in envelope_lv2:
        envelope_lv3['general_parts'] = get_general_parts(envelope_lv2['general_parts'], common_lv2)
        
    if 'windows' in envelope_lv2:
        envelope_lv3['windows'] = get_windows(envelope_lv2['windows'], common_lv2)
        
    if 'doors' in envelope_lv2:
        envelope_lv3['doors'] = get_doors(envelope_lv2['doors'], common_lv2)
        
    if 'heatbridges' in envelope_lv2:
        envelope_lv3['heatbridges'] = get_heatbridges(envelope_lv2['heatbridges'], common_lv2)
        
    if 'earthfloor_perimeters' in envelope_lv2:
        envelope_lv3['earthfloor_perimeters'] = get_earthfloor_perimeters(envelope_lv2['earthfloor_perimeters'])
        
    if 'earthfloor_centers' in envelope_lv2:
        envelope_lv3['earthfloor_centers'] = get_earthfloor_centers(envelope_lv2['earthfloor_centers'])

    return {
        'common'   : common_lv3,
        'envelope' : envelope_lv3,
    }

example

In [18]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputUA',                     
                    'UA': 0.24,
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllDetails',
                    'IsInContactWithOutsideAir': True, 
                    'Parts': [{'AreaRatio': 0.8, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'AreaRatio': 0.2, 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                            {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False 
                }
            },
            {
                'name'       : 'floor',
                'general_part_type'  : 'floor',
                'next_space' : 'outdoor',
                'direction'  : 'bottom',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllLayers',
                    'IsInContactWithOutsideAir': True, 
                    'ConstructionMethodAllLayers' :'FloorFrameBeamInsuljoist',
                    'Parts': [{'Type': 'FloorInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                     {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'Type': 'FloorHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                     {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'wall',
                'general_part_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'n',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllLayers', 
                    'IsInContactWithOutsideAir': True,
                    'ConstructionMethodAllLayers': 'WallFrameInsulcolumn',
                    'Parts': [{'Type': 'WallInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                    {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'Type': 'WallHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                    {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllLayers', 
                    'IsInContactWithOutsideAir': True,
                    'ConstructionMethodAllLayers': 'RoofInsulRafter', 
                    'Parts': [{'Type': 'RoofInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                    {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'Type': 'RoofHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                    {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False 
                }
            },
            {
                'name'       : 'boundary_ceiling',
                'general_part_type'  : 'boundary_ceiling',
                'next_space' : 'air_conditioned',
                'direction'  : 'upward',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood',
                    'InputMethod': 'InputAllLayers', 
                    'IsInContactWithOutsideAir': True,
                    'ConstructionMethodAllLayers': 'CeilingInsulbeam', 
                    'Parts': [{'Type': 'CeilingInsulation', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                       {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]},
                              {'Type': 'CeilingHeatbridge', 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                                                       {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False
                }
            },
            
                        {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                }
            },
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'wood', 
                    'InputMethod': 'InputUR', 
                    'InsulationMethod': 'CeilingInsulbeam',
                    'IsInContactWithOutsideAir': True,              
                    'Parts': [{ 'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                           {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],
                    'IsSunshadeInput': False 
                }
            },
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'RC', 
                    'InputMethod': 'InputUA', 
                    'UA': 0.24, 
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'RC',
                    'InputMethod': 'InputLayers', 
                    'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'steel',
                    'InputMethod': 'InputUA', 
                    'UA': 0.24,
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'steel',
                    'InputMethod': 'InputUR', 
                    'IsInContactWithOutsideAir': True,
                    'area': 67.8,
                    'Radding': 0.10, 
                    'Parts': [{'Layers': [{'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 },
                                          {'name': 'wood','thick': 0.012, 'cond': 0.16, 'specH': 720 }]}],         
                    'IsSunshadeInput': False 
                }
            },
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure': 'other',
                    'InputMethod': None, 
                    'IsSunshadeInput': False 
                }
            },
        ],
        'windows': [
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                # spec の中身は後日要見直し
                'spec'       : {
                    'UW': 6.51,
                    'TypeWindow': 'Single', 
                    'EtaInputMethod': 'InputValue',
                    'TypeFrame': 'WoodOrResin',
                    'TypeGlass': '3WgG',
                    'TypeShade': 'Shoji',
                    'IsSunshadeInput': True,
                    'Y1': 0.00,
                    'Y2': 1.00,
                    'Z': 0.60
                }
            },
            {
                'name'       : 'window_nw', 
                'next_space' : 'outdoor', 
                'direction'  : 'nw', 
                'area'       : 3.17, 
                # spec の中身は後日要見直し
                'spec'       : {
                    'UW': 4.65, 
                    'TypeWindow': 'Single', 
                    'EtaInputMethod': 'InputValue',
                    'Eta': 0.738, 
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                # spec の中身は後日要見直し
                'spec'       : {
                    'UW': 6.51,
                    'TypeWindow' : 'Double', 
                    'EtaInputMethod': 'InputSpecification',
                    'TypeFrameInside': 'WoodOrResin',
                    'TypeGlassInside': '3WgG',
                    'TypeShadeInside': 'Shoji',
                    'TypeFrameOutside': 'Steel', 
                    'TypeGlassOutside': '3WgG', 
                    'TypeShadeOutside': 'ExtarnalBlind',
                    'IsSunshadeInput': True, 
                    'Y1': 0.00,
                    'Y2': 1.00,
                    'Z': 0.60
                }
            },
            { 
                'name'       : 'window_nw',
                'next_space' : 'outdoor', 
                'direction'  : 'nw',
                'area'       : 3.17,
                # spec の中身は後日要見直し
                'spec'       : {
                    'UW': 4.65,
                    'TypeWindow': 'Double', 
                    'EtaInputMethod': 'InputValue',
                    'EtaInside': 0.738,
                    'TypeGlassInside': '3WgG',
                    'EtaOutside': 0.738, 
                    'TypeGlassOutside': '3WgG',
                    'IsSunshadeInput': True,
                    'Y1': 0.00,
                    'Y2': 1.00, 
                    'Z': 0.60
                }
            }
        ],
        'doors': [
            {
                'name'       : 'door_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 2.52,
                # spec の中身は後日要見直し
                'spec'       : {
                    'U'               : 6.51,
                    'IsSunshadeInput' : False  
                }
            },
            {
                'name'       : 'door_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'area'       : 2.16,
                # spec の中身は後日要見直し
                'spec'       : {
                    'U': 4.65,
                    'IsSunshadeInput': True,
                    'Y1': 0.00,
                    'Y2': 1.00,
                    'Z': 0.60
                }
            }
        ],
        'heatbridges': [
            {
                'name'       : 'heatbridge_ne',
                'length'     : 1.00,
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'e'],
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure' : 'steel',
                    'psi'       : 1.8,
                }
            },
            {
                'name'       : 'heatbridge_nw',
                'length'     : 2.00,
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'w'],
                # spec の中身は後日要見直し
                'spec'       : {
                    'structure' : 'steel',
                    'psi'       : 1.8,
                }
            },
        ],
        'earthfloor_perimeters': [
            {
                'name'       : 'earthfloor_perimeter_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'length'     : 2.43,
                'spec'       : {
                    'psi': 1.8,
                }
            },
            {
                'name'       : 'earthfloor_perimeter_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'length'     : 2.43,
                'spec'       : {
                    'psi': 1.8,
                }
            },
        ],
        'earthfloor_centers': [
            { 'area': 5.0, 'name': 'other' },
            { 'area': 5.0, 'name': 'entrance' }
        ],
    }
}

convert(d)

{'common': {'region': 6,
  'main_occupant_room_floor_area': 30.0,
  'other_occupant_room_floor_area': 60.0,
  'total_floor_area': 120.0},
 'envelope': {'general_parts': [{'name': 'ceiling_main_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'main_occupant_room',
    'spec': {'structure': 'wood',
     'InputMethod': 'InputUA',
     'UA': 0.24,
     'IsSunshadeInput': False}},
   {'name': 'ceiling_other_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 33.9,
    'space_type': 'other_occupant_room',
    'spec': {'structure': 'wood',
     'InputMethod': 'InputUA',
     'UA': 0.24,
     'IsSunshadeInput': False}},
   {'name': 'ceiling_non_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'non_occupant_room',
    'spec': {'structure': 'wood',


In [19]:
d = {
    'common':{
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 30.0,
        'total_floor_area': 90.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'direction'  : 'top',
                'area'       : 50.85,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 7.7,'is_sunshade_input': False}
            },
            {
                'name'       : 'wall_sw',
                'general_part_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.47,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 6.67,'is_sunshade_input': False}
            },
            {
                'name'       : 'wall_nw',
                'general_part_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 22.37,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 6.67,'is_sunshade_input': False}
            },
            {
                'name'       : 'wall_ne',
                'general_part_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'area'       : 47.92,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 6.67,'is_sunshade_input': False}
            },
            {
                'name'       : 'wall_se',
                'general_part_type'  : 'wall',
                'next_space' : 'outdoor',
                'direction'  : 'se',
                'area'       : 22.28,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 6.67,'is_sunshade_input': False}
            },
            {
                'name'       : 'floor',
                'general_part_type'  : 'floor',
                'next_space' : 'opened_underfloor',
                'direction'  : 'bottom',
                'area'       : 45.05,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 5.27,'is_sunshade_input': False}
            },
            {
                'name'       : 'floor_bath',
                'general_part_type'  : 'floor',
                'next_space' : 'opened_underfloor',
                'direction'  : 'bottom',
                'area'       : 3.31,
                'spec'       : {'structure':'wood','u_value_input_method_wood':'u_value_directly','u_value': 5.27,'is_sunshade_input': False}
            },
        ],
        'windows': [
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 22.69,
                # spec の中身は後日要見直し
                'spec'       : {
                    'TypeWindow': 'Single',
                    'EtaInputMethod': 'InputValue', 
                    'etaCooling': 0.51,
                    'etaHeating': 0.51,
                    'UW': 3.49,
                    'IsSunshadeInput': True,
                    'Y1': 0, 
                    'Y2': 1.1, 
                    'z': 0.348
                }
            },
            {
                'name'       : 'window_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw', 
                'area'       : 2.38,
                # spec の中身は後日要見直し
                'spec'       : {
                    'TypeWindow': 'Single',
                    'EtaInputMethod': 'InputValue', 
                    'etaCooling': 0.51,
                    'etaHeating': 0.51,
                    'UW': 3.49,
                    'IsSunshadeInput': True,
                    'Y1': 0, 
                    'Y2': 1.1, 
                    'z': 0.348
                }
            },
            {
                'name'       : 'window_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne', 
                'area'       : 3.63,
                # spec の中身は後日要見直し
                'spec'       : {
                    'TypeWindow': 'Single',
                    'EtaInputMethod': 'InputValue', 
                    'etaCooling': 0.51,
                    'etaHeating': 0.51,
                    'UW': 3.49,
                    'IsSunshadeInput': True,
                    'Y1': 0, 
                    'Y2': 1.1, 
                    'z': 0.348
                }
            },
            {
                'name'       : 'window_se',
                'next_space' : 'outdoor',
                'direction'  : 'se', 
                'area'       : 4.37,
                # spec の中身は後日要見直し
                'spec'       : {
                    'TypeWindow': 'Single',
                    'EtaInputMethod': 'InputValue', 
                    'etaCooling': 0.51,
                    'etaHeating': 0.51,
                    'UW': 3.49,
                    'IsSunshadeInput': True,
                    'Y1': 0, 
                    'Y2': 1.1, 
                    'z': 0.348
                }
            }
        ],
        'doors': [
            {
                'name'       : 'door_nw', 
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 1.89,
                # spec の中身は後日要見直し
                'spec'       : {
                    'U': 4.65,
                    'IsSunshadeInput': False
                }
            },
            {
                'name'       : 'door_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'area'       : 1.62,
                # spec の中身は後日要見直し
                'spec'       : {
                    'U': 4.65,
                    'IsSunshadeInput': False
                }
            }
        ],
        'EarthfloorPerimeters': [
            {
                'name'       : 'earthfloor_perimeter_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'length'     : 1.82,
                'spec'       : {
                    'psi': 1.8,
                }
            },
            {
                'name'       : 'earthfloor_perimeter_ne',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'length'     : 1.37,
                'spec'       : {
                    'psi': 1.8,
                }
            },
            {
                'name'       : 'earthfloor_perimeter_entrance',
                'next_space' : 'open_underfloor',
                'direction'  : 'nw',
                'length'     : 3.19,
                'spec'       : {
                    'psi': 1.8,
                }
            },
        ],
    }
}

convert(d)

{'common': {'region': 6,
  'main_occupant_room_floor_area': 30.0,
  'other_occupant_room_floor_area': 30.0,
  'total_floor_area': 90.0},
 'envelope': {'general_parts': [{'name': 'ceiling_main_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'main_occupant_room',
    'spec': {'structure': 'wood',
     'u_value_input_method_wood': 'u_value_directly',
     'u_value': 7.7,
     'is_sunshade_input': False}},
   {'name': 'ceiling_other_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'other_occupant_room',
    'spec': {'structure': 'wood',
     'u_value_input_method_wood': 'u_value_directly',
     'u_value': 7.7,
     'is_sunshade_input': False}},
   {'name': 'ceiling_non_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type